In [40]:
import numpy as np
import pandas as pd
import sys, os

### Scheduling Analysis

In [340]:
df_logtime4 = pd.read_csv("./logTime_4runtimes_2.csv")
df_stoic = pd.read_csv("./logTime_stoic_2.csv")

In [341]:
# Check if 4 runtimes are in line with each workload
df_edge = df_logtime4[df_logtime4["runtime"]=="edge"]
for index, row in df_edge.iterrows():
    if index % 4 != 0:
        print (index)
        print (df_logtime4[index-4:index+4])
        sys.exit(0)

df_edge.reset_index(inplace=True)

In [342]:
df_stoic_front = df_stoic[0:len(df_edge)]

In [343]:
df_edge[df_stoic_front["image_num"] != df_edge["image_num"]]

,index,epoch,image_num,runtime,pred_total,pred_transfer,pred_deploy,pred_proc,act_total,act_transfer,act_deploy,act_proc


In [344]:
runtimes = []
for index in range(0, len(df_logtime4), 4):
    minTime = float("inf")
    runtime = ""
    for i in range(index, index + 4):
        if (df_logtime4[i: i+1]["act_total"].item() < minTime):
            minTime = df_logtime4[i: i + 1]["act_total"].item()
            runtime = df_logtime4[i: i + 1]["runtime"].item()
    runtimes.append(runtime)
    

In [345]:
groundtruth_runtimes = pd.Series(runtimes)
stoic_runtimes = df_stoic_front["runtime"]
positive = stoic_runtimes[stoic_runtimes == groundtruth_runtimes]

In [346]:
len(positive) / len(df_stoic_front)

0.6576576576576577

In [347]:
groundtruth_runtimes

0      edge
1      gpu2
2      gpu1
3      gpu2
4      gpu2
5      edge
6      gpu2
7      gpu2
8      gpu2
9      gpu2
10     gpu2
11     edge
12     edge
13     edge
14     gpu2
15     gpu2
16     gpu2
17     edge
18     edge
19     edge
20     edge
21     gpu1
22     gpu2
23     gpu2
24     edge
25     edge
26     edge
27     gpu2
28     gpu1
29     gpu2
       ... 
192    gpu1
193    gpu1
194    gpu1
195    edge
196    edge
197    edge
198    gpu1
199    gpu2
200    gpu1
201    edge
202    edge
203    edge
204    gpu1
205    gpu1
206    gpu1
207    gpu2
208    gpu1
209    edge
210    edge
211    edge
212    edge
213    gpu1
214    gpu1
215    gpu1
216    gpu1
217    edge
218    edge
219    gpu1
220    gpu2
221    edge
Length: 222, dtype: object

In [348]:
stoic_runtimes

0      edge
1      gpu2
2      gpu2
3      edge
4      gpu2
5      edge
6      gpu2
7      gpu2
8      gpu2
9      gpu2
10     gpu2
11     edge
12     gpu2
13     edge
14     gpu2
15     gpu2
16     gpu2
17     gpu2
18     edge
19     edge
20     gpu2
21     gpu2
22     gpu2
23     gpu2
24     edge
25     edge
26     gpu2
27     gpu2
28     gpu2
29     gpu2
       ... 
192    gpu2
193    gpu2
194    gpu2
195    edge
196    edge
197    gpu2
198    gpu2
199    gpu2
200    gpu2
201    edge
202    edge
203    edge
204    gpu2
205    gpu2
206    gpu2
207    gpu2
208    gpu2
209    edge
210    edge
211    gpu2
212    gpu2
213    gpu2
214    gpu2
215    gpu2
216    gpu2
217    edge
218    gpu2
219    gpu2
220    gpu2
221    edge
Name: runtime, Length: 222, dtype: object

### MSR

In [335]:
sum((df_stoic["pred_total"] - df_stoic["act_total"])**2) / len(df_stoic)

597.9151582357928

### MAE

In [336]:
sum(abs(df_stoic["pred_total"] - df_stoic["act_total"])) / len(df_stoic)

9.616440786003297

### GPU acceleration analysis

In [324]:
counter = 0
for index in range(2, len(df_logtime4), 4):
    if df_logtime4[index:index+1]["act_total"].item() < df_logtime4[index+1:index+2]["act_total"].item():
            counter += 1
print (counter)
print (counter / (len(df_logtime4)/4))

95
0.42792792792792794


In [279]:
df_gpu = pd.read_csv("./logTime_gpu_2.csv")

In [301]:
df_gpu[190:192]

,epoch,image_num,runtime,pred_total,pred_transfer,pred_deploy,pred_proc,act_total,act_transfer,act_deploy,act_proc
190,1580370138,150,gpu1,97.30056,10.552733,15.44597,71.301857,105.081322,10.552733,33.948799,71.132523
191,1580370333,150,gpu2,97.30056,10.552733,15.44597,71.301857,140.154465,10.552733,67.745750,72.408707


In [299]:
counter = 0
for index in range(0, len(df_gpu), 2):
    if df_gpu[index:index+1]["act_proc"].item() < df_gpu[index+1:index+2]["act_proc"].item():
        print (df_gpu[index:+index+1])
        print (df_gpu[index+1:+index+2])
        counter += 1
print ("Speed up count : ", counter)
print ("Speed up rate : ", round(counter/(len(df_gpu)/2), 2))

        epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
6  1580346955         10    gpu1   47.752846       0.599278     15.44597   

   pred_proc  act_total  act_transfer  act_deploy   act_proc  
6  31.707598  63.747971      0.599278   30.437233  33.310738  
        epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
7  1580347107         10    gpu2   47.752846       0.599278     15.44597   

   pred_proc  act_total  act_transfer  act_deploy   act_proc  
7  31.707598   95.90052      0.599278    42.58799  53.312531  
        epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
8  1580347252         10    gpu1   47.767311       0.613744     15.44597   

   pred_proc  act_total  act_transfer  act_deploy   act_proc  
8  31.707598   88.50676      0.613744   55.125984  33.380779  
        epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
9  1580347385         10    gpu2   47.767311       0.613744     15.44597   

   pred_pr

164  52.458321  94.342987      7.584836    42.55389  51.789093  
          epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
165  1580366430        100    gpu2   75.489128       7.584836     15.44597   

     pred_proc  act_total  act_transfer  act_deploy   act_proc  
165  52.458321  82.377304      7.584836   27.512337  54.864964  
          epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
166  1580366550        100    gpu1   75.810722       7.584836     15.44597   

     pred_proc  act_total  act_transfer  act_deploy   act_proc  
166  52.779919  73.533394      7.584836   21.503477  52.029915  
          epoch  image_num runtime  pred_total  pred_transfer  pred_deploy  \
167  1580366722        100    gpu2   75.810722       7.584836     15.44597   

     pred_proc   act_total  act_transfer  act_deploy   act_proc  
167  52.779919  115.306992      7.584836   57.571182  57.735809  
          epoch  image_num runtime  pred_total  pred_transfer  pred_depl

In [ ]:
df_gpu